# Ledger Signed

Like simple ledger, but here each person must use they key to sign that the transaction should take place. So we don't have any one person making things up.

You'll need to install `ecdsa` for this

In [16]:
import ecdsa
from ecdsa import SigningKey, VerifyingKey, SECP256k1
import hashlib

In [17]:
def gen_private_public_key_pair():
	private_key = SigningKey.generate(curve=SECP256k1)
	public_key = private_key.get_verifying_key()
	return private_key, public_key

private, public = gen_private_public_key_pair()
print(private, public)

<ecdsa.keys.SigningKey object at 0x10a55aea0> VerifyingKey.from_string(b'\x02~\x0e\xd4\x85\xa2\xdcq\xe0\x12\xb4\xfb_\x11\xa7mo\xd2]\x12\x0f\x92\xe4[Y\xbe;\xbd\xfdI\xcf\x8b\xf2', SECP256k1, sha1)


In [34]:
def crypto_hash(_bytes):
	return hashlib.sha256(_bytes).digest()

crypto_hash("Hello there!!!".encode())

b'm\x8bj\xc5\xf99\xf1\x85?\\U\xe6\xfa\xc9\x92\r\xefw\xfa\x94\xc8/M\xecg#R\x9bnC\xcf+'

In [43]:
def sign(private_key, data):
	return private_key.sign(data)
def verify(public_key, signature, data):
	try: 
		public_key.verify(signature, data)
		return True
	except Exception:
		return False

data = "hello!".encode()
signature = sign(private, data)
verify(public, signature, data)

True

Same thing but based on hashed value

In [44]:
hashed = crypto_hash("hello!".encode())
signature = sign(private, data)
verify(public, signature, data)

True

## Signed transactions in central ledger

In [45]:
class Wallet:
	def __init__(self, id, starting_amount=0):
		self.cash = starting_amount
		self.id = id
		self._private_key, self.public_key = gen_private_public_key_pair()

	def __repr__(self):
		return f"Wallet(id='{self.id}', cash=${self.cash})"

alice = Wallet("alice", 10)
bob = Wallet("bob", 15)
carl = Wallet("carl", 10)
donny = Wallet("donny", 5)

print(alice, bob, carl, donny)

Wallet(id='alice', cash=$10) Wallet(id='bob', cash=$15) Wallet(id='carl', cash=$10) Wallet(id='donny', cash=$5)


In [49]:
class Ledger:
	def __init__(self, init_wallets: list[Wallet]):
		self.gid = 0
		self.history = []
		for w in init_wallets:
			self.add_transaction_to_history(source=None, dest=w.id, amount=w.cash, source_private_key=w._private_key)
			
	def add_transaction_to_history(self, source, dest, amount, source_private_key):
		transaction = {"gid": self.gid, "from": source, "to": dest, "amount": amount}
		hashed = crypto_hash(str(transaction).encode())
		signature = sign(source_private_key, hashed)
		self.history.append((transaction, hashed, signature))
		self.gid += 1
	
	def wallet_amount(self, wallet: Wallet):
		id = wallet.id
		summed  = 0
		for (t, hashed, signature) in self.history:
			if t["from"] == id:
				if verify(wallet.public_key, signature, hashed):
					# Sent money to someone else
					summed -= t["amount"]
			elif t["to"] == id:
					# Money sent to us
					summed += t["amount"]
		return summed



	def transact(self, a: Wallet, b: Wallet, amount):
		# first check if a has enough money to send to b 
		a_cash = self.wallet_amount(a)
		assert a_cash >= amount, "Source needs enough money to send to destination"

		# if we are here, we can execute the transaction
		self.add_transaction_to_history(a.id, b.id, amount, a._private_key)

l = Ledger([alice, bob, carl, donny])	

l.transact(alice, donny, 10)

print(l.wallet_amount(alice), l.wallet_amount(donny))

l.transact(donny, alice, 5)
print(l.wallet_amount(alice), l.wallet_amount(donny))

0 15
5 10
